In [2]:
import numpy as np
import math
import matplotlib.pyplot as plt  
from sklearn.metrics import mean_squared_error, mean_absolute_error
import pandas as pd


import sys
import xlwt
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
import datetime,pickle,os,glob



def transfromData(trainRaw, testRaw,windosSize):  ##Train ratial, train, test
    sc = MinMaxScaler(feature_range = (0, 1))
    #npRaw = np.array(rawData)
    #scaledData = sc.fit_transform(npRaw.reshape(-1,1))  
    #trainData = np.array(scaledData[:trainCount]).reshape(-1,1)##train 
    #testData = np.array(scaledData[-testCount:]).reshape(-1,1)
    
    npTrain = sc.fit_transform(np.array(trainRaw).reshape(-1,1))
    npTest = sc.fit_transform(np.array(testRaw).reshape(-1,1))
    

    X_train, y_train = splitXy(npTrain,windosSize)
    X_test, y_test = splitXy(npTest,windosSize)
    return sc, X_train, y_train, X_test, y_test

def splitXy(data,windosSize):
    windows = []
    for i in range(windosSize, data.shape[0]):
        windows.append(data[i-windosSize:i, 0])
    np.random.shuffle(windows)
    X = []
    y = []
    for i in range(len(windows)):
        X.append(windows[i][:6])
        y.append(windows[i][-(windosSize-6):])
    X, y = np.array(X), np.array(y)
    #X = np.reshape(X, (X.shape[0], X.shape[1], 1))
    return X,y




def buildModel():
    dnn_1_input = Input(shape=(6,),name='dnn_1_input')
    dnn_1 = Dense(128,activation="tanh", name='dnn_1')(dnn_1_input)
    dnn_2 = Dense(64,activation="tanh", name='dnn_2')(dnn_1)
    dnn_3 = Dense(32,activation="tanh", name='dnn_3')(dnn_2)
    dnn_4 = Dense(1, activation='tanh', name='dnn_4')(dnn_3)

    model = Model(dnn_1_input,dnn_4)
    model.summary()
    model.compile(loss='mse',
                  optimizer='adam')
    return model
def getStationList():
    with open('pickles/stationList.pickle', 'rb') as handle:
        stationList = pickle.load(handle)
    os.chdir('excelFiles/LSTM')
    replaceDict = ['.xls','LSTMresult']
    for direct in glob.glob("*.xls"):
        fileName = direct                                               
        for w in replaceDict:
            fileName = fileName.replace(w,'')
        
        if fileName in stationList:
            stationList.remove(fileName)
            print(fileName)
    os.chdir('../..')
    return stationList

def writeExcelHead(sheet1,epochs,station):
    sheet1.write(0,1,station)
    raw = 1
    for e in range(epochs):
        sheet1.write(raw,0,e+1)
        raw+=1
def fetchData(station,windosSize):

    with open('pickles/'+station+'2017trainRaw.pickle', 'rb') as handle:
        trainRawData = pickle.load(handle)
    with open('pickles/'+station+'2017testRaw.pickle', 'rb') as handle:
        testRawData = pickle.load(handle)
        
    sc, X_train, y_train, X_test, y_test = transfromData(trainRawData,testRawData,windosSize)
    return sc, X_train, y_train, X_test, y_test
def trainLR(sc, X_train, y_train, X_test, y_test):
    reg = LinearRegression().fit(X_train, y_train)
    predicted_LR = sc.inverse_transform(reg.predict(X_test))
    ori_y_test = sc.inverse_transform(y_test)
    mse = mean_squared_error(predicted_LR, ori_y_test)
    mae = mean_absolute_error(predicted_LR, ori_y_test)
    print(mse)
    return mse, mae

In [3]:


stationList = getStationList()
col=1
for station in stationList:
    print("training : " +station)
    mseList = []
    maeList = []
    for windowSize in range(7,31):
        sc, X_train, y_train, X_test, y_test = fetchData(station,windowSize)
        mse, mae = trainLR(sc, X_train, y_train, X_test, y_test)
        mseList.append(mse)
        maeList.append(mae)
    book = xlwt.Workbook(encoding="utf-8")
    sheet1 = book.add_sheet("Sheet1")
    writeExcelHead(sheet1,10,station)
    row = 1
    for mse in mseList:
        sheet1.write(row,1,mse)
        row+=1
    row = 1
    for mae in maeList:
        sheet1.write(row,2,mae)
        row+=1
    book.save("excelFiles/LR/LRresult"+station+".xls")


training : 二林
24.67836090372834
38.52563500414973
53.60278095440413
65.04802261015915
74.67271637440561
82.87533296638969
89.86799687347987
96.05761448877695
101.57382337312542
106.4469643405549
110.68417021997742
114.3685427328777
117.67539588728076
120.78832632304254
123.65667352905275
126.36432908277494
128.8857042658219
131.2658493465332
133.47306281164498
135.57300254599892
137.51947424604683
139.3687744066837
141.10923933885093
142.7924413367888
training : 南投
15.574554692608313
30.322600204810012
42.59784225867759
53.27123872588286
62.57729642483967
70.63109405137878
77.69644254540944
83.99306223196959
89.71852156267549
94.89157358010719
99.44026982791819
103.50166331707824
107.27438715367647
110.83448469557204
114.19234466470857
117.28946843299975
120.17012192306083
122.8586025945195
125.35849876625133
127.72167032917564
129.96097565815214
132.12496824237735
134.22631902680766
136.2655877821076
training : 埔里
12.842479261260248
27.652286186124098
40.489426426396506
52.10420827119

32.78253691073392
44.76007563997234
53.10294842733731
59.92911512586875
65.71097875060379
70.6631562718394
75.05753517179157
79.04022205870268
82.65821639757264
85.96104297406606
88.99369257790819
91.73069470303412
94.18212850387165
96.40096418341798
98.39753562037981
100.21252491058608
101.85978274798615
103.36501945013089
104.7642115825328
106.08568478937835
107.33680547423337
108.5202971350071
109.6456257525041
training : 大園
66.30941575613852
101.65624089868916
141.9995911625121
168.05587648482793
189.3882071567119
208.67813241911406
226.66285578121577
243.93398340351087
260.7417276688933
277.25220588364226
293.455167253185
309.3562018659623
324.9369554272017
340.1283185510933
354.8371360144594
369.0722862647035
382.88230029925285
396.2406891328648
409.16167657342925
421.67330406141826
433.84532503656146
445.69341398756393
457.21289088078464
468.45027306100366
training : 平鎮
20.157623290753044
29.552793674961073
41.0487641707553
48.53557403782442
54.73809536580977
60.131527470763416


40.986750504732704
43.156616705774304
45.071763792003225
46.770803090776965
48.284974327687316
49.629911455137744
50.839448010620636
51.93007842148601
52.89570998748875
53.75231900811246
54.5221126816119
55.20385615684022
55.809936440224405
56.36093514564963
56.86822263639452
57.354877190628756
57.83308721601939
58.31182143548077
training : 宜蘭
7.107907503846565
12.386673579620735
16.030301496903302
19.090345200644737
21.74234122504865
24.04277787792201
26.0692478103469
27.860697589490215
29.45639492735111
30.88684409518317
32.1720546258906
33.33147203761518
34.386447972754425
35.37605921326376
36.30931397432514
37.17514246626336
37.95960986327322
38.67262755008824
39.32421706727825
39.93540091524301
40.5080054926911
41.03923301844284
41.531913085018736
42.00099865436034
training : 三義
13.73449491994213
26.064335753995362
35.02646120629177
42.589951539974635
49.08427849887752
54.6898124573158
59.571804925284866
63.909415863187675
67.79368916446849
71.30419410867593
74.4316439916278
77.19

117.28122691183721
121.59994057360274
125.43665179152056
128.87246569907228
131.87432276544638
134.51004734166506
136.79998684787645
138.87444659847213
140.72748352770603
142.41321442136518
143.95498491058547
145.41329402877307
146.83594775289606
148.23953323183468
149.67470575816256
training : 崙背
32.492654764205405
50.5655694528439
72.426270215028
88.74678383272592
102.67406784869414
114.95557396014135
125.85138636927034
135.6696892216228
144.47014051351044
152.25624688199284
159.0358550462063
164.80291892699262
169.97057476413147
174.50764598952713
178.67601116809232
182.37844564966906
185.71679259113003
188.708780186474
191.32693510245232
193.74660721086659
195.98719288085894
198.1281247015824
200.1765764628601
202.1341665076994
training : 斗六
40.678371467507674
62.043467342760934
86.6650129992729
103.88731582253146
118.19766260897836
130.3669686655538
140.64064709570349
149.5318893870209
157.2820347767559
164.0038446149616
169.81066268719917
174.97657792816037
179.62553029296623
183

90.12460052291489
97.13280651147038
103.374535197894
108.95138067732778
113.7959113047737
118.20958195863979
122.11842610872958
125.64694616736745
128.69500930585448
131.2420122438867
133.37350263121712
135.12197631252536
136.56962777019888
137.8427733826921
139.00472452739663
140.08354568938637
141.13577047840667
142.1777422139544
training : 潮州
52.23076207536572
84.18265275007307
119.12212361294227
142.21204839697268
159.4639580380309
173.08596901909996
184.52094467609624
194.2154315290593
202.4922569965875
209.44144108905022
215.19574709887453
219.92177536208808
223.7449454915272
226.84235346320114
229.36428481511138
231.30138799816922
232.70961690733182
233.68595942801747
234.3722533378462
234.95359546041558
235.53205140491684
236.17061246524142
236.9237893928268
237.8151867461818
training : 美濃
24.875225401650077
30.549570170904023
35.394281391264286
39.71875370288962
43.441547811580264
46.80864898522757
49.79412902120306
52.43756123296883
54.84460518831445
57.13483052442414
59.2505

In [4]:
k = [1,2,3,4,5,6,7,8,9]
k[-6:]

[4, 5, 6, 7, 8, 9]

In [10]:
k = [1,2,3,4,5]
k[2:]



[3, 4, 5]

In [ ]:
        X.append(windows[i][:6])
        y.append(windows[i][-(windosSize-6):])